<a href="https://colab.research.google.com/github/ipin131/NLP-Myanimelist-Review-Singeky-no-kyojin/blob/main/Assignment_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium


In [ ]:
#Hanya untuk 1 halaman

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL target
url = "https://myanimelist.net/anime/16498/Shingeki_no_Kyojin/reviews"

# Fungsi untuk scraping data ulasan
def scrape_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = []

    # Cari setiap ulasan berdasarkan kategori
    for review_section in soup.find_all('div', class_='review-element'):
        try:
            # Ambil kategori ulasan (Recommended, Mixed Feelings, Not Recommended)
            category = review_section.find('div', class_='tag').get('data-id')
            # Ambil teks ulasan
            text = review_section.find('div', class_='text').get_text(strip=True)
            reviews.append({'Category': category, 'Review': text})
        except AttributeError:
            continue

    return reviews

# Scrape dan simpan ke DataFrame
data = scrape_reviews(url)
df = pd.DataFrame(data)

# Simpan ke CSV
df.to_csv("reviews.csv", index=False)
print("Data berhasil disimpan ke 'reviews.csv'")


Data berhasil disimpan ke 'reviews.csv'


In [ ]:
# 3 Halaman review

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fungsi untuk scraping data ulasan dari satu halaman
def scrape_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = []

    # Cari setiap ulasan berdasarkan kategori
    for review_section in soup.find_all('div', class_='review-element'):
        try:
            # Ambil kategori ulasan (Recommended, Mixed Feelings, Not Recommended)
            category = review_section.find('div', class_='tag').get('data-id')
            # Ambil teks ulasan
            text = review_section.find('div', class_='text').get_text(strip=True)
            reviews.append({'Category': category, 'Review': text})
        except AttributeError:
            continue

    return reviews

# URL target untuk halaman 1, 2, dan 3
urls = [
    "https://myanimelist.net/anime/16498/Shingeki_no_Kyojin/reviews?sort=suggested&filter_check=&filter_hide=&preliminary=on&spoiler=off&p=1",
    "https://myanimelist.net/anime/16498/Shingeki_no_Kyojin/reviews?sort=suggested&filter_check=&filter_hide=&preliminary=on&spoiler=off&p=2",
    "https://myanimelist.net/anime/16498/Shingeki_no_Kyojin/reviews?sort=suggested&filter_check=&filter_hide=&preliminary=on&spoiler=off&p=3"
]

# Gabungkan semua ulasan dari halaman-halaman yang berbeda
all_reviews = []
for url in urls:
    all_reviews.extend(scrape_reviews(url))

# Simpan ke DataFrame
df = pd.DataFrame(all_reviews)

# Simpan ke CSV
df.to_csv("reviews.csv", index=False)
print("Data berhasil disimpan ke 'reviews.csv'")


Data berhasil disimpan ke 'reviews.csv'


In [ ]:
!pip install nltk
import nltk
nltk.download('punkt_tab')
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Preprocessing function
def preprocess_text(text):
    # Hapus simbol dan angka
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Lowercasing
    text = text.lower()
    # Tokenisasi
    tokens = word_tokenize(text)
    # Hapus stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Terapkan preprocessing ke data
df['Processed_Review'] = df['Review'].apply(preprocess_text)

# Simpan hasil preprocessing
df.to_csv("reviews_preprocessed.csv", index=False)
print("Data preprocessed berhasil disimpan ke 'reviews_preprocessed.csv'")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Data preprocessed berhasil disimpan ke 'reviews_preprocessed.csv'


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Load the dataset
csv_path = "reviews_preprocessed.csv"
df = pd.read_csv(csv_path)

# Ensure the required columns exist
if 'Processed_Review' in df.columns and 'Category' in df.columns:
    # Step 2: Vektorisasi data ulasan
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['Processed_Review'])
    y = df['Category']

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 4: Train and evaluate Naive Bayes model
    nb_model = MultinomialNB()
    nb_model.fit(X_train, y_train)
    nb_preds = nb_model.predict(X_test)

    # Step 5: Train and evaluate SVM model
    svm_model = SVC(kernel='linear')
    svm_model.fit(X_train, y_train)
    svm_preds = svm_model.predict(X_test)

    # Step 6: Evaluation metrics
    print("Naive Bayes Report:")
    print(classification_report(y_test, nb_preds))

    print("SVM Report:")
    print(classification_report(y_test, svm_preds))
else:
    print("Required columns 'Processed_Review' and 'Category' are missing in the CSV.")


Naive Bayes Report:
              precision    recall  f1-score   support

           1       0.60      0.86      0.71         7
           2       1.00      0.20      0.33         5
           3       0.00      0.00      0.00         0

    accuracy                           0.58        12
   macro avg       0.53      0.35      0.35        12
weighted avg       0.77      0.58      0.55        12

SVM Report:
              precision    recall  f1-score   support

           1       0.67      0.86      0.75         7
           2       0.67      0.40      0.50         5

    accuracy                           0.67        12
   macro avg       0.67      0.63      0.62        12
weighted avg       0.67      0.67      0.65        12



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
